In [9]:
%run ./setup.ipynb

You should consider upgrading via the 'C:\Users\alibo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\alibo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


 This code :
* Reads a large CSV file (data.csv)
* Cleans and processes the data to remove any null values.
* Saves the data to an AWS S3 bucket (cleaned_data.csv).
* AWS Secrets are stored in the notebook above


The aforementioned input file in spite of its size is assumed to still fit into my laptop memory. It can be processed using pandas to cleanse and remove nulls as well as duplicatess

In [2]:
import pandas as pd
import boto3
import logging
import os
import json
from datetime import datetime
BUCKET='mys3bucket'

In [3]:
#=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
def process_file(input_file,output_file,bucket,object_name):
    try:
        #1.loading CSV file into pandas dataframe
        df=pd.read_csv(input_file)
        #2. clean data (remove duplicates and null values)
        df.dropna(how='any',axis=0, inplace=True)
        df.dropna(axis=1,how='all',inplace=True)
        print(datetime.now().strftime("[%Y-%m-%d][%H:%M:%S:%f] "+"Input file cleansed"))
        #3. writing pandas dataframe into a file 
        df.to_csv(output_file)
        print(datetime.now().strftime("[%Y-%m-%d][%H:%M:%S:%f] "+"Output file created successfully!"))
        #4. Upload the file to s3
        session = session = boto3.Session(
            aws_access_key_id=AWS_ACCESS_KEY_ID,
            aws_secret_access_key=AWS_SECRET_ACCESS_KEY
        )
        print(datetime.now().strftime("[%Y-%m-%d][%H:%M:%S:%f] "+"Authenticating access to S3..."))
        s3 = session.resource('s3')
        print(datetime.now().strftime("[%Y-%m-%d][%H:%M:%S:%f] "+"Uploading file into S3..."))
        response = s3.Bucket(BUCKET).upload_file(output_file, object_name)
    except Exception as e:
        print(datetime.now().strftime("[%Y-%m-%d][%H:%M:%S:%f] "+str(e)))
        return False
    return True

Below is a sample code to call the function:

In [4]:
input_file='data.csv'
output_file='cleansed_data.csv'
bucket=BUCKET
success=process_file(input_file,output_file,bucket, output_file)
if(success==True):
    print(datetime.now().strftime("[%Y-%m-%d][%H:%M:%S:%f] "+"File uploaded to S3 Successfuly!"))

[2024-07-30][10:16:29:074792] Input file cleansed
[2024-07-30][10:16:29:077753] Output file created successfully!
[2024-07-30][10:16:29:090059] Authenticating access to S3...
[2024-07-30][10:16:29:457280] Uploading file into S3...
[2024-07-30][10:16:30:383002] File uploaded to S3 Successfuly!


For processing larger data sets particularly when it comes to data volume rather than number of records, parallel processing frameworks such as spark is recommended. Spark loads big data set into memory, distributes it across a set of worker nodes and each worker node partitions the data to process. Number of worker nodes or executors can be configured and increased or decreased based on workload and available resources to enhance scalabiity.